In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
import pyodbc
import gradio as gr

In [89]:
credential = AzureKeyCredential("dc15ce3d9f5b43578eea933f1c609939")
endpoint="https://cognitive363.cognitiveservices.azure.com/"
text_analytics_client = TextAnalyticsClient(endpoint, credential)

In [90]:
server = 'sqlserver363.database.windows.net'
database = 'server363'
username = 'sqlserver363'
password = '{Abcd1234}'
driver= '{ODBC Driver 17 for SQL Server}'

In [91]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

In [ ]:
documents = ["I did not like the restaurant. The food was somehow both too spicy and underseasoned. Additionally, I thought the location was too far away from the playhouse."]

In [95]:
def sentiment_analysis(hotelname, name, surname, gender, comment):
    documents = []
    documents.append(comment)
    response = text_analytics_client.analyze_sentiment(documents, language="en")
    result = [doc for doc in response if not doc.is_error]

    for doc in result:
        if doc.sentiment == "positive":
            label = 1
            return ("We're thrilled to hear that you enjoyed your trip! Thank you for your positive feedback!")
        if doc.sentiment == "negative":
            label = -1
            return ("We're sorry to hear that! We're doing our best to avoid negative experiences and will continue to do so. Hope to see you on your next trip!")
        if doc.sentiment == "neutral":
            label = ()
            return ("Thank you for your valuable feedback!")
    cursor = cnxn.cursor()
    cursor.execute("""insert into gradio_table values(%s, %s, %s, %s, %s, %s);""",(name, surname, gender, comment, label, hotelname,))
    cursor.commit()

In [ ]:
iface = gr.Interface(sentiment_analysis,
                     [
        gr.inputs.Dropdown(["The Savoy Hotel", "Fairmont Hotel", "The Stanley Hotel"], type="index", label="Select hotel"),
        gr.inputs.Textbox(label="Name"),
        gr.inputs.Textbox(label="Surname"),
        gr.inputs.Radio(["Female", "Male", "I don't want to share / I don't identify as either"], type="index",  label="Select gender"),
        gr.inputs.Textbox(label="Comment"),
    ],
                     title=["Share your comment!"], 
                     description=["Please let us know of your comments so we can improve ourselves!"], 
                     outputs=["text"])

iface.launch(debug = True)

Running on local URL:  http://127.0.0.1:7879/

To create a public link, set `share=True` in `launch()`.
